In [2]:
import json
from tqdm import tqdm
with open('data.json') as f:
        data = json.load(f)
print(len(data))

705


In [7]:
def get_synonym(disease, disease_description):
    ''' Return list of synonym
    Args:
        - disease (str) : non-acent
        - disease-description (list) of sentences
    '''
    # get vietnamese disease
    vi_disease = get_vi_disease(disease,disease_description)
    
    # get synonyms : E
    # TODO
    
    return vi_disease

def find_synonyms(disease):
    '''
    - main type
    - related type
    - synonym 1-1
    - english
    Args:
        - disease (str)
    '''
def remove_accent(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s

def get_vi_disease(disease,disease_description):
    disease = disease.replace('benh ','').strip()
    # find disease in non-accent description
    for sent in disease_description:
        sent_ = remove_accent(sent.lower())
        start_idx = sent_.find(disease)
        if start_idx != -1:
            return sent[start_idx : start_idx + len(disease)].lower()
    return disease

In [11]:
# Unique key
import hashlib
import re

def get_unique_key(value):
    ''' Return a hash for given dict
    Args:
        - value (dict)
    '''
    return hashlib.sha1(json.dumps(value, sort_keys=True).encode()).hexdigest()

In [12]:
DISEASE = []
for sample in data:
    disease = get_vi_disease(sample['disease'][0],sample['disease-description'])
    if disease == sample['disease'][0]
    
    DISEASE.append(disease)
print(len(DISEASE))

705


In [13]:
DISEASE[:10]

['rối loạn tiền đình',
 'lao phoi',
 'te bi tay chan',
 'viêm đại tràng',
 'giời leo',
 'uốn ván',
 'áp xe',
 'viêm xoang',
 'sốt rét',
 'suy thận']

In [52]:
err = []
for i, d in enumerate(DISEASE):
    if d == remove_accent(d):
        err.append((i,d))
print(len(err))
# lao
# tê bì chân tay
# virus ebola
# alzheimer
# Addison (suy tuyến thượng thận nguyên phát)
# alkapton niệu
# lao phổi
# Lang ben
# basedow
# Beriberi
# Sa sút trí tuệ Dementia
# Bệnh quai bị
# bệnh sởi
# bệnh Rubella
# brucella
# Babesia
# Cryptosporidium
# crohn
# Căng cơ
# Chấy rận , chí rận
# chốc đầu, nấm da đầu
# Chửa ở vết mổ

129


In [99]:
data[err[23][0]]

{'disease': ['chan an than kinh bieng an tam ly'],
 'disease-description': ['Những người bị các chứng rối loạn ăn uống có những hành vi ăn uống rối loạn và niềm tin méo mó với những nỗi lo lắng thái quá về cân nặng, hình dáng, ăn uống và hình ảnh cơ thể. Các chứng rối loạn ăn uống là các chứng bệnh tâm thần nghiêm trọng; đó không phải là một lựa chọn về lối sống.',
  'Các chứng rối loạn ăn uống không chỉ bao gồm sự suy sụp về tâm lý và nỗi đau khổ mà còn đi kèm với những biến chứng y tế lớn trên phạm vi rộng và nghiêm trọng mà có thể gây ảnh hưởng đến tất cả các bộ phận quan trọng trong cơ thể.',
  'Các chứng rối loạn ăn uống xảy ra cả ở nam giới và nữ giới, người giàu và người nghèo, và từ tất cả các văn hóa. Không ai muốn mình bị rối loạn ăn uống cả. ',
  'Chán ăn tâm thần (tên tiếng Anh là Anorexia Nervosa) thuộc 1 trong 3  chứng rối loạn cho ăn và ăn uống đã 

In [46]:
from mtranslate import translate
EN_DISEASE = []
for d in tqdm(DISEASE):
    translation = translate(d, 'en')
    EN_DISEASE.append(translation)
print(len(EN_DISEASE))
print(EN_DISEASE[:3])

100%|██████████| 705/705 [05:47<00:00,  2.03it/s]

705
['vestibular disorders', 'tuberculosis', 'te bi tay chan']


In [17]:
import argparse
import os
import pdb
import pickle
from BioSyn.src.biosyn import (
    DictionaryDataset,
    BioSyn,
    TextPreprocess
)

class EntityLinker:
    def __init__(self,model_name_or_path,use_cuda=False):
        self.model_name_or_path = model_name_or_path
        self.biosyn = BioSyn(
            max_length=25,
            use_cuda=use_cuda
        )
        self.biosyn.load_model(model_name_or_path=model_name_or_path)

    def cache_or_load_dictionary(self, dictionary_path):
        dictionary_name = os.path.splitext(os.path.basename(dictionary_path))[0]

        cached_dictionary_path = os.path.join(
            './tmp',
            f"cached_{self.model_name_or_path.split('/')[-1]}_{dictionary_name}.pk"
        )

        # If exist, load the cached dictionary
        if os.path.exists(cached_dictionary_path):
            with open(cached_dictionary_path, 'rb') as fin:
                cached_dictionary = pickle.load(fin)
            print("Loaded dictionary from cached file {}".format(cached_dictionary_path))

            dictionary, dict_sparse_embeds, dict_dense_embeds = (
                cached_dictionary['dictionary'],
                cached_dictionary['dict_sparse_embeds'],
                cached_dictionary['dict_dense_embeds'],
            )

        else:
            dictionary = DictionaryDataset(dictionary_path = dictionary_path).data
            dictionary_names = dictionary[:,0]
            dict_sparse_embeds = self.biosyn.embed_sparse(names=dictionary_names, show_progress=True)
            dict_dense_embeds = self.biosyn.embed_dense(names=dictionary_names, show_progress=True)
            cached_dictionary = {
                'dictionary': dictionary,
                'dict_sparse_embeds' : dict_sparse_embeds,
                'dict_dense_embeds' : dict_dense_embeds
            }

            if not os.path.exists('./tmp'):
                os.mkdir('./tmp')
            with open(cached_dictionary_path, 'wb') as fin:
                pickle.dump(cached_dictionary, fin)
            print("Saving dictionary into cached file {}".format(cached_dictionary_path))

        return dictionary, dict_sparse_embeds, dict_dense_embeds
    
    def __call__(self,mention,dictionary_path):

        # preprocess mention
        mention = TextPreprocess().run(mention)

        # embed mention
        mention_sparse_embeds = self.biosyn.embed_sparse(names=[mention])
        mention_dense_embeds = self.biosyn.embed_dense(names=[mention])

        output = {
            'mention': mention,
        }
        # cache or load dictionary
        dictionary, dict_sparse_embeds, dict_dense_embeds = self.cache_or_load_dictionary(dictionary_path)

        # calcuate score matrix and get top 5
        sparse_score_matrix = self.biosyn.get_score_matrix(
            query_embeds=mention_sparse_embeds,
            dict_embeds=dict_sparse_embeds
        )
        dense_score_matrix = self.biosyn.get_score_matrix(
            query_embeds=mention_dense_embeds,
            dict_embeds=dict_dense_embeds
        )
        sparse_weight = self.biosyn.get_sparse_weight().item()
        hybrid_score_matrix = sparse_weight * sparse_score_matrix + dense_score_matrix
        hybrid_candidate_idxs = self.biosyn.retrieve_candidate(
            score_matrix = hybrid_score_matrix, 
            topk = 5
        )

        # get predictions from dictionary
        predictions = dictionary[hybrid_candidate_idxs].squeeze(0)
        output['predictions'] = []

        for prediction in predictions:
            predicted_name = prediction[0]
            predicted_id = prediction[1]
            output['predictions'].append({
                'name': predicted_name,
                'id': predicted_id
            })

        return output

In [18]:
Entity_Linker = EntityLinker(
    model_name_or_path="dmis-lab/biosyn-sapbert-ncbi-disease",
    use_cuda=True )

/home/minh/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 0.24.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/minh/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.24.2 when using version 0.24.0. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [21]:
dictionary_path = "BioSyn/datasets/ncbi-disease/test_dictionary.txt"
related_entities = Entity_Linker("Sinusitis",dictionary_path)

/home/minh/work/knowledge_base_system/BioSyn/src/biosyn/biosyn.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
embedding dictionary: 100%|██████████| 72/72 [01:07<00:00,  1.07it/s]


Saving dictionary into cached file ./tmp/cached_biosyn-sapbert-ncbi-disease_test_dictionary.pk


In [100]:
related_entities

{'mention': 'sinusitis',
 'predictions': [{'name': 'sinusitis', 'id': 'D012852'},
  {'name': 'sinusitides', 'id': 'D012852'},
  {'name': 'maxillary sinusitis', 'id': 'D015523'},
  {'name': 'sinusitis maxillary', 'id': 'D015523'},
  {'name': 'maxillary sinusitides', 'id': 'D015523'}]}

In [105]:
EL_DATA = []

for disease_object, vi_disease, en_disease in tqdm(zip(data,DISEASE,EN_DISEASE)):
    d_object = {}
    d_object['key'] = get_unique_key(disease_object)

    for k,v in disease_object.items():
        d_object[k] = v
                
    en_entities = Entity_Linker(en_disease,dictionary_path)
    
    synonym_disease = []
    en_entities = [item['name'] for item in en_entities['predictions']]
    synonym_disease.extend(d_object['disease'])
    synonym_disease.extend(en_entities)
    synonym_disease.extend([vi_disease])
    
    synonym_diseas = list(set(synonym_disease))
    
    d_object['synonyms'] = synonym_disease
    
    EL_DATA.append(d_object)

1it [00:00,  3.86it/s]

Loaded dictionary from cached file ./tmp/cached_biosyn-sapbert-ncbi-disease_test_dictionary.pk


2it [00:00,  3.88it/s]

Loaded dictionary from cached file ./tmp/cached_biosyn-sapbert-ncbi-disease_test_dictionary.pk


3it [00:00,  3.90it/s]

...

Loaded dictionary from cached file ./tmp/cached_biosyn-sapbert-ncbi-disease_test_dictionary.pk


704it [03:02,  3.82it/s]

Loaded dictionary from cached file ./tmp/cached_biosyn-sapbert-ncbi-disease_test_dictionary.pk


705it [03:03,  3.85it/s]

Loaded dictionary from cached file ./tmp/cached_biosyn-sapbert-ncbi-disease_test_dictionary.pk


In [106]:
with open('el_data.json', 'w') as f:
    json.dump(EL_DATA, f, ensure_ascii=False)